In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import gc
import albumentations as A

In [26]:
#----------------------Inception model handling ---------------
import tensorflow as tf
import sys
sys.path.append('C:/Users/aadis/Documents/Anaconda WorkSpace/TensorFlow/Universal OpenImage Predictor/models/research/slim/')
from nets import inception_v3

model = inception_v3.inception_v3()

TypeError: inception_v3() missing 1 required positional argument: 'inputs'

In [ ]:
def classify(image_path):
    
    # Use the Inception model to classify the image.
    pred = model.classify(image_path=image_path)

    # Print the scores and names for the top-10 predictions.
    model.print_scores(pred=pred, k=10, only_first_name=True)

In [2]:
df = pd.read_csv('E:/Datasets/validation-annotations-human-imagelabels.csv', usecols=[0,2,3])

In [3]:
df = df[df.Confidence == 1]

In [4]:
df.LabelName.unique()

array(['/m/0k5j', '/m/015y8h', '/m/018rqw', ..., '/m/04p33s', '/m/0c40t',
       '/m/06_knc'], dtype=object)

In [6]:
classes = np.array(['/m/01g317', '/m/09j2d', '/m/04yx4', '/m/0dzct', '/m/07j7r', '/m/05s2s', '/m/03bt1vf', '/m/07yv9', '/m/0cgh4', '/m/01prls', '/m/09j5n', '/m/0jbk', '/m/0k4j', '/m/05r655', '/m/02wbm', '/m/0c9ph5', '/m/083wq', '/m/0c_jw', '/m/03jm5', '/m/0d4v4'])

#**************Notes***********
#/m/01g317 : Bittern
#/m/09j2d    Clothing
#/m/04yx4
#/m/0dzct    Human face
#/m/07j7r   Tree
#/m/05s2s    Plant
#/m/03bt1vf
#/m/07yv9    Vehicle
#/m/0cgh4     Building
#'/m/01prls', Land vehicle 
#'/m/09j5n'   Footwear
#'/m/0jbk'    Animal
#'/m/0k4j',    Car
# '/m/05r655'
#'/m/02wbm',   Food
#'/m/0c9ph5'   Flower
#'/m/083wq'    Wheel
#/m/0c_jw'     Furniture
#'/m/03jm5'    House
#'/m/0d4v4'    Window
#####################################

In [7]:
li = []
for i in classes:
    li.append(df[df.LabelName == i])
df = pd.concat(li).sample(frac=1).reset_index(drop=True)
del li
gc.collect()
df.head()

,ImageID,LabelName,Confidence
0,8b349b63e4385558,/m/0k4j,1
1,1158ff693584049b,/m/07yv9,1
2,1eb5447743b21204,/m/02wbm,1
3,9822a048d45b5139,/m/01prls,1
4,2a19b4586ac1052e,/m/07yv9,1


In [8]:
labels = df.LabelName.tolist()
Imageid = df.ImageID.values
print(len(df))

68867


In [9]:
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Conv2D, PReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model, Sequential
from tqdm import tqdm_notebook
from tqdm import tqdm
from keras.optimizers import Adam

Using TensorFlow backend.


In [10]:
df.head(),gc.collect()

(            ImageID  LabelName  Confidence
 0  8b349b63e4385558    /m/0k4j           1
 1  1158ff693584049b   /m/07yv9           1
 2  1eb5447743b21204   /m/02wbm           1
 3  9822a048d45b5139  /m/01prls           1
 4  2a19b4586ac1052e   /m/07yv9           1, 0)

In [11]:
X_train = [np.array(load_img('E:/Datasets/validation/{}.jpg'.format(i),target_size=(100,100), grayscale=True))/255 for i in tqdm(Imageid[10000:11000])]

  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\aadis\Anaconda3\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 1000/1000 [01:36<00:00, 12.63it/s]


In [12]:
X_Val = [np.array(load_img('E:/Datasets/validation/{}.jpg'.format(i),target_size=(100,100), grayscale=True))/255 for i in tqdm(Imageid[:200])]

  0%|          | 0/200 [00:00<?, ?it/s]C:\Users\aadis\Anaconda3\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 200/200 [00:15<00:00, 10.69it/s]


In [13]:
classes = classes.tolist()
Y_train, Y_Val = [], []
for i in tqdm(labels[10000:11000]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_train.append(temp)
    del temp
for i in tqdm(labels[:200]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_Val.append(temp)
    del temp
Y_train = np.array(Y_train)
Y_Val = np.array(Y_Val)
gc.collect(), Y_train[0]

100%|██████████| 200/200 [00:00<00:00, 200157.67it/s]


(9, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

In [14]:
nn = Sequential()
nn.add(BatchNormalization(input_shape=(100, 100, 1)))
nn.add(Conv2D(4, kernel_size=(2,2), strides=(1,1)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(8, kernel_size=(2,2), strides=(1,1)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(16, kernel_size=(2,2), strides=(2,2)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(32, kernel_size=(2,2), strides=(1,1)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(32, kernel_size=(2,2), strides=(2,2)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(32, kernel_size=(2,2), strides=(2,2)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Flatten())
nn.add(Dense(2048))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(1024))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(512))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(128))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(50))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(25))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(20, activation='softmax'))

In [15]:
nn.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'], optimizer='adam')

In [16]:
X_train = np.array(X_train).reshape((1000,100,100,1))
X_Val = np.array(X_Val).reshape((200,100,100,1))

In [17]:
gc.collect()

0

In [18]:
nn.fit(X_train, Y_train, validation_data=(X_Val,Y_Val), batch_size=100, epochs=5, verbose=2)

Train on 1000 samples, validate on 200 samples
Epoch 1/5
 - 217s - loss: 3.6521 - acc: 0.0500 - val_loss: 3.3259 - val_acc: 0.0250
Epoch 2/5
 - 198s - loss: 3.5306 - acc: 0.0520 - val_loss: 3.5755 - val_acc: 0.0150
Epoch 3/5
 - 197s - loss: 3.4668 - acc: 0.0670 - val_loss: 3.7252 - val_acc: 0.0350
Epoch 4/5
 - 202s - loss: 3.3994 - acc: 0.0710 - val_loss: 4.2698 - val_acc: 0.0400
Epoch 5/5
 - 203s - loss: 3.2725 - acc: 0.0720 - val_loss: 6.0686 - val_acc: 0.0600


In [19]:
del X_train, Y_train, X_Val, Y_Val, df
gc.collect()

74377

In [20]:
df = pd.read_csv('E:/Datasets/all/stage_1_sample_submission.csv', usecols=[0])
im = df.image_id.tolist()
df.head()

,image_id
0,2b2b327132556c767a736b3d
1,2b2b394755692f303963553d
2,2b2b42584e6d445937444d3d
3,2b2b44744e57674270616f3d
4,2b2b4b425a592b683059493d


In [21]:
for i in tqdm_notebook(im):
    try: 
        X_test = np.array(load_img('E:/Datasets/all/stage_1_test_images/{}.jpg'.format(i),target_size=(100,100), grayscale=True))/255
    except:
        pass

C:\Users\aadis\Anaconda3\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [22]:
X_test = np.array(X_test).reshape((1,100,100,1))

In [23]:
pre = nn.predict(X_test).argsort(1)[:,:5]
del X_test
gc.collect()

0

In [24]:
p = []
print(pre)
for it in tqdm(pre):
    p.append([classes[int(i)] for i in it])

[[ 1 16  8  5  6]]


100%|██████████| 1/1 [00:00<?, ?it/s]


In [25]:
print(p)

[['/m/09j2d', '/m/083wq', '/m/0cgh4', '/m/05s2s', '/m/03bt1vf']]


In [26]:
print(p[0])

['/m/09j2d', '/m/083wq', '/m/0cgh4', '/m/05s2s', '/m/03bt1vf']


In [27]:
p1 = str(p[0])
print(p1)

['/m/09j2d', '/m/083wq', '/m/0cgh4', '/m/05s2s', '/m/03bt1vf']


In [28]:
class_description = pd.read_csv('E:/Datasets/all/class-descriptions.csv', usecols=[0,1])
label_list = class_description.label_code.tolist()

for i in p[0]:
    for k in range(0,len(class_description.label_code)):
        j = class_description.label_code[k]
        if i == j:
            print(j)
            print(class_description.description[k])
            break;

/m/09j2d
Clothing
/m/083wq
Wheel
/m/0cgh4
Building
/m/05s2s
Plant


In [ ]:
LightAug = Compose([
      CLAHE(),
      HorizontalFlip(.5),
      ShiftScaleRotate(shift_limit=0.075, scale_limit=0.15, rotate_limit=10, p=.75 ),
      Blur(blur_limit=3, p=.33),
      OpticalDistortion(p=.33),
      GridDistortion(p=.33),
      HueSaturationValue(p=.33)
  ], p=p)